# 0. Import dependencies

In [6]:
# libraries
import cv2
import numpy as np
import torch
import os

import sys
sys.path.append('swinface_project/')
# packages

from model import build_model

import warnings
warnings.filterwarnings('ignore')

# 1. Face Alignment Phrase


## 1.1 Alignment base opencv

In [2]:
from imutils import face_utils
import argparse
import imutils
import dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('aligment/shape_predictor_68_face_landmarks.dat')

def rotate(right_eye, left_eye, image):
    midpoint = ((left_eye[0] + right_eye[0]) / 2.0, (left_eye[1] + right_eye[1]) / 2.0)
    angle = np.arctan2(right_eye[1] - left_eye[1], right_eye[0] - left_eye[0]) * 180 / np.pi
    midpoint = tuple(midpoint)
    rotation_matrix = cv2.getRotationMatrix2D(midpoint, angle, scale=1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
    return rotated_image,rotation_matrix
def rotate_point(point,rotation_matrix):
    # Convert coordinates to NumPy arrays for convenience
    point = np.array(point)

    # Add a row [0, 0, 1] to represent homogeneous coordinates
    point_homogeneous = np.append(point, 1)

    # Apply the rotation matrix
    rotated_point_homogeneous = np.dot(rotation_matrix, point_homogeneous)

    # Convert back to coordinates without homogeneous coordinates
    rotated_point = rotated_point_homogeneous[:2]

    return tuple(map(int, rotated_point))

In [54]:
def alignment_opencv(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    # if detector more than one face
    if len(rects) !=1 :
        return None
    shape = predictor(gray, rects[0])
    shape = face_utils.shape_to_np(shape)
    right_eye = shape[25]
    left_eye = shape[20]
    rotate_img= rotate(right_eye,left_eye,image)
    right_eye = shape[25]
    left_eye = shape[20]
    rotate_img,rotate_matrix = rotate(right_eye,left_eye,image)
    x_max = shape[0][0]
    y_max = shape[0][1]
    x_min = shape[0][0]
    y_min = shape[0][1]
    for (x,y) in shape:
        (x,y) = rotate_point((x,y),rotate_matrix)
        if x_max < x:
            x_max = x
        elif x_min > x:
            x_min = x
        elif y_max < y:
            y_max = y
        elif y_min > y:
            y_min = y
    crop_img = rotate_img[y_min-60:y_max+60,x_min-40:x_max+40]
    return crop_img


In [55]:
path_img = 'img/008.jpg'
img = cv2.imread(path_img)
alignment_img = alignment_opencv(img)
cv2.imwrite("img/alignment_img_008.jpg",alignment_img)


True

## 1.2 Alignment base Star (HairFastGan)

# 2. Face Analysis Phrase

## 2.1. Init parameters

In [3]:
class SwinFaceCfg:
    network = "swin_t"
    fam_kernel_size=3
    fam_in_chans=2112
    fam_conv_shared=False
    fam_conv_mode="split"
    fam_channel_attention="CBAM"
    fam_spatial_attention=None
    fam_pooling="max"
    fam_la_num_list=[2 for j in range(11)]
    fam_feature="all"
    fam = "3x3_2112_F_s_C_N_max"
    embedding_size = 512
    
cfg = SwinFaceCfg()

## 2.2. Create/load swin-face model

In [4]:
# create_swinface_model
def create_swinface_model(cfg, weight_path):
    model = build_model(cfg)
    dict_checkpoint = torch.load(weight_path)
    model.backbone.load_state_dict(dict_checkpoint["state_dict_backbone"])
    model.fam.load_state_dict(dict_checkpoint["state_dict_fam"])
    model.tss.load_state_dict(dict_checkpoint["state_dict_tss"])
    model.om.load_state_dict(dict_checkpoint["state_dict_om"])

    model.eval()
    return model 

weight_path = "swinface_project/checkpoint_step_79999_gpu_0.pt"
swin_face_model = create_swinface_model(cfg, weight_path)

## 2.3. Inference on one image

In [ ]:
@torch.no_grad()
def inference(model, img):
    if img is None:
        img = np.random.randint(0, 255, size=(112, 112, 3), dtype=np.uint8)
    else:

        img = cv2.resize(img, (112, 112))

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.transpose(img, (2, 0, 1))
    img = torch.from_numpy(img).unsqueeze(0).float()
    img.div_(255).sub_(0.5).div_(0.5)
    
    output = model(img)#.numpy()

    return output["Recognition"]

image = cv2.imread("img/001.jpg")
facial_embedding = inference(swin_face_model, image)
facial_embedding

## 2.4. Inference on images

In [59]:
# facial embedding for all images in directory (jpg and png)
def inference_in_dir_path(model, dir_path):
    files = os.listdir(dir_path)
    files = [f for f in files if f.endswith(".jpg") or f.endswith(".png")]

    embeddings = []
    for f in files:
        img = cv2.imread(os.path.join(dir_path, f))
        facial_embedding = inference(model, img)
        embeddings.append(facial_embedding)

    return files, torch.stack(embeddings, dim=0)

dir_path = "img"
list_of_image, embeddings = inference_in_dir_path(swin_face_model, dir_path)
embeddings.shape

torch.Size([5, 1, 512])

## 2.5. Caculate the cosine similarity between two embedding

In [60]:
# caculate similarity between 2 embeddings (torch.tensor)
def similarity(embedding1, embedding2):
    return torch.nn.functional.cosine_similarity(embedding1, embedding2)

# caculate similarity between embeddings and a specific embedding
def similarity_with_specific_embedding(embeddings, specific_embedding):
    list_of_scores = []
    for i in range(embeddings.shape[0]):
        sim = similarity(embeddings[i], specific_embedding)
        list_of_scores.append(sim)
    return list_of_scores
    
sim_scores = similarity_with_specific_embedding(embeddings, embeddings[1])
sim_scores

[tensor([0.2042]),
 tensor([1.0000]),
 tensor([0.1023]),
 tensor([0.1435]),
 tensor([0.3496])]